In [143]:
import pandas as pd
import json
import requests
import datetime as dt
import pandas_datareader as pdr

In [144]:
security_breaches = pd.read_csv('/Users/garimajain/Desktop/SecurityBreaches-7.csv')

In [145]:
df1 = security_breaches.iloc[1:]

In [146]:
breach_data = df1.to_csv("Lab3")

In [147]:
breaches = pd.read_csv('/Users/garimajain/Desktop/Lab3.csv')

In [186]:
def get_symbol(name):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(name)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
            return x['symbol']
        
ticker = []
company = breaches.Entity
for  symbol in company:
    ticker.append(get_symbol(symbol))

In [187]:
ticker_df = pd.DataFrame(ticker)
new_data = breaches.assign(Symbol = ticker)

In [150]:
data_breach = new_data.dropna(axis=0,how='any')

In [151]:
new_data.to_csv('Lab3_with_ticker.csv')

In [189]:
#getting stock price data for the available ticker's data
start_date = dt.datetime(2004,1,1)
end_date = dt.datetime(2017,6,6)
data_source = 'yahoo'
panel_data = pdr.DataReader(ticker, data_source, start_date, end_date)

In [153]:
panel_data

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3652 (major_axis) x 64 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2004-01-01 00:00:00 to 2017-06-07 00:00:00
Minor_axis axis: None to ^UMIAMIFL

In [154]:
close = panel_data.ix['Close']

In [155]:
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

In [156]:
close = close.reindex(all_weekdays)

In [157]:
revised_data = close.dropna(axis=1,how='all')

In [173]:
avg_by_month = revised_data.groupby(pd.TimeGrouper(freq='1M')).mean()

In [174]:
data_rearrangemet = avg_by_month.T

In [175]:
#created a dataframe for de-normalized data about Ticker symbol, date and corresponding stock price
final_data = pd.DataFrame(columns=['Symbol','Date','Price'])
d =0
for i in range(len(data_rearrangemet)) : 
    ts = data_rearrangemet.index[i]
    for j in range(len(data_rearrangemet.iloc[i])): 
        timet = pd.to_datetime(str(data_rearrangemet.columns.values[j]))
        date = timet.strftime('%Y-%m-%d')
        final_data.loc[d]=( ts,  date , data_rearrangemet.iloc[i][j])
        d=d+1

In [179]:
final_data.to_csv('Final_Lab3_new.csv')